***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [25]:
import numpy as np
import pandas as pd
import re
import nltk
nltk.download('punkt')
from nltk.tokenize import wordpunct_tokenize, word_tokenize, sent_tokenize, TreebankWordTokenizer
import gensim
from gensim.utils import tokenize
from gensim.models.keyedvectors import KeyedVectors

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ivano\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
from gensim.test.utils import datapath

In [3]:
wv_embeddings = KeyedVectors.load_word2vec_format(datapath("SO_vectors_200.bin"), binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [9]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [10]:
print(f"Num of words: {len(wv_embeddings.index_to_key)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли слов `cat` топ-5 близких слов к слову `dog`? Какое место?

In [11]:
wv_embeddings.most_similar('dog', topn = 5)

[('animal', 0.8564180135726929),
 ('dogs', 0.7880866527557373),
 ('mammal', 0.7623804211616516),
 ('cats', 0.7621253728866577),
 ('animals', 0.760793924331665)]

4 место

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [12]:
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ivano\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ivano\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [17]:
lemmatizer = WordNetLemmatizer()

class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        token = re.findall('[a-zA-Z0-9#+]+', text)
        txt = [lemmatizer.lemmatize(word) for word in token
                if not word in set(stopwords.words('english'))]
        return txt
tokenizer = MyTokenizer()

reg_tokenizer = RegexpTokenizer(r'[a-zA-Z0-9#+]+')

In [15]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
    Преобразует вопрос в векторное представление как среднее векторов известных слов.

    :param question: строка
    :param embeddings: словарь с векторными представлениями слов
    :param tokenizer: функция токенизации
    :param dim: размер любого вектора в нашем представлении
    :return: векторное представление для вопроса
    """
    
    tokens = tokenizer.tokenize(question)
    q_vector = np.zeros(dim)  # Инициализация итогового вектора

    valid_count = 0  # Счетчик известных слов
    for word in tokens:
        if word in embeddings:
            q_vector += embeddings[word]  # Суммируем векторы
            valid_count += 1  # Увеличиваем счетчик известных слов

    if valid_count > 0:
        final_vector = q_vector / valid_count  # Средний вектор
    else:
        final_vector = np.zeros(dim)  # Если нет известных слов

    return final_vector


Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [18]:
text = 'I love neural networks'
question_to_vec(text, wv_embeddings, reg_tokenizer)[2].round(2)

-1.29

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями.

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?

**Ответ**: 1, так как максимальное значение Hits@47 это 1, а минимальное значение DCG@1 это 0

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера.
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

In [19]:
rank = 9
k = 10
round(1/np.log2(1+9), 1) if rank <=k else 0

0.3

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [20]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """
    sum = 0
    for rank in dup_ranks:
      if rank <= k:
        sum+=1
    return sum/len(dup_ranks)

In [21]:
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
    """
    sum = 0
    for rank in dup_ranks:
      if rank <= k:
        sum+=1/np.log2(1+rank)
    return sum/len(dup_ranks)

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [22]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наги кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks_func = lambda x : [i+1 for i, val in enumerate(x) if val in copy_answers]
dup_ranks = dup_ranks_func(candidates_ranking[0])

In [23]:
# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [27]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [28]:
# для выполнения локально
import zipfile
 
with zipfile.ZipFile('./stackoverflow_similar_questions.zip', 'r') as zip_ref:
    zip_ref.extractall('./unpacked_files')

Считайте данные.

In [29]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        data.append(line.strip().split('\t'))
    return data

Нам понадобиться только файл validation.

In [30]:
validation_data = read_corpus('./unpacked_files/data/validation.tsv')

Кол-во строк

In [31]:
len(validation_data)

3760

Размер нескольких первых строк

In [20]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [32]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [33]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    questionVector = question_to_vec(question, embeddings, tokenizer) #Векторное представление вопроса
    candidatesVectors = np.array([question_to_vec(candidate, embeddings, tokenizer) for candidate in candidates]) #Получаем векторное представление для кандидатов
    similarities = cosine_similarity(questionVector.reshape(1, -1), candidatesVectors)
    sorted_indices = np.argsort(similarities[0])[::-1] # Получаем индексы кандидатов, отсортированные по убыванию косинусного сходства
    ranked_candidates = [(index, candidates[index]) for index in sorted_indices] # Формируем результат в виде списка пар (позиция, кандидат)

    return ranked_candidates

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [34]:
questions = ['converting string to list', 'Sending array via Ajax fails']

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],

              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [35]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer=reg_tokenizer)
        print(ranks)
        print()

[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]

[(1, 'WPF- How to update the changes in list item of a list'), (0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results')]



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

102

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [36]:
# для выполнения в vscode требуется другой формат импорта
import tqdm

In [37]:
validation_data[0][:5]

['How to print a binary heap tree without recursion?',
 'How do you best convert a recursive function to an iterative one?',
 'How can i use ng-model with directive in angular js',
 'flash: drawing and erasing',
 'toggle react component using hide show classname']

In [39]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm.tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer=reg_tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

 27%|██▋       | 1000/3760 [00:23<01:06, 41.76it/s]


In [40]:
for k in tqdm.tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

100%|██████████| 6/6 [00:00<00:00, 999.12it/s]

DCG@   1: 0.235 | Hits@   1: 0.235
DCG@   5: 0.291 | Hits@   5: 0.343
DCG@  10: 0.308 | Hits@  10: 0.395
DCG@ 100: 0.354 | Hits@ 100: 0.622
DCG@ 500: 0.380 | Hits@ 500: 0.822
DCG@1000: 0.398 | Hits@1000: 1.000


### Эмбеддинги, обученные на корпусе похожих вопросов

In [41]:
train_data = read_corpus('./unpacked_files/data/train.tsv')

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

In [42]:
len(train_data)

1000000

In [43]:
train_data[:5]

[['converting string to list',
  'Convert Google results object (pure js) to Python object'],
 ['Which HTML 5 Canvas Javascript to use for making an interactive drawing tool?',
  'Event handling for geometries in Three.js?'],
 ['Sending array via Ajax fails',
  'Getting all list items of an unordered list in PHP'],
 ['How to insert CookieCollection to CookieContainer?',
  'C# create cookie from string and send it'],
 ['Updating one element of a bound Observable collection',
  'WPF- How to update the changes in list item of a list']]

In [44]:
tokenizer = RegexpTokenizer(r'[a-zA-Z0-9#+]+')
lemmatizer = WordNetLemmatizer()
for i in range(0, len(train_data)):
    txt = ' '.join(train_data[i]).lower()
     
    token = tokenizer.tokenize(txt)
    # loop for stemming each word ; replacing train_data
    txt = [lemmatizer.lemmatize(word) for word in token
                if not word in set(stopwords.words('english'))]
    train_data[i] = txt

In [45]:
train_data[:5]

[['converting',
  'string',
  'list',
  'convert',
  'google',
  'result',
  'object',
  'pure',
  'j',
  'python',
  'object'],
 ['html',
  '5',
  'canvas',
  'javascript',
  'use',
  'making',
  'interactive',
  'drawing',
  'tool',
  'event',
  'handling',
  'geometry',
  'three',
  'j'],
 ['sending',
  'array',
  'via',
  'ajax',
  'fails',
  'getting',
  'list',
  'item',
  'unordered',
  'list',
  'php'],
 ['insert',
  'cookiecollection',
  'cookiecontainer',
  'c#',
  'create',
  'cookie',
  'string',
  'send'],
 ['updating',
  'one',
  'element',
  'bound',
  'observable',
  'collection',
  'wpf',
  'update',
  'change',
  'list',
  'item',
  'list']]

In [46]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(train_data, # data for model to train on
                 sg= 1,
                 vector_size =200,               # embedding vector size
                 min_count=5,
                 alpha = 0.01,                 
                 window=15, 
                 epochs = 7).wv

In [47]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm.tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer = MyTokenizer())
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

 27%|██▋       | 1000/3760 [29:33<1:21:33,  1.77s/it]


In [49]:
for k in tqdm.tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

100%|██████████| 6/6 [00:00<00:00, 999.16it/s]

DCG@   1: 0.274 | Hits@   1: 0.274
DCG@   5: 0.340 | Hits@   5: 0.397
DCG@  10: 0.361 | Hits@  10: 0.462
DCG@ 100: 0.397 | Hits@ 100: 0.636
DCG@ 500: 0.417 | Hits@ 500: 0.800
DCG@1000: 0.438 | Hits@1000: 1.000


### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:


1) Токенизация без учёта знаков препинания, так как задача посвящена анализу предложений, а не целого текста, где пунктуация необходима для определения контекста предложений. split по пробелу, например, также создает отдельные токены для слов, представленных в скобках. Кроме того, с помощью regular expressions были сохранены # и +, так как необходимы в комбинациях C++, С#
2) Да, применение методов нормализации повышает качество модели, так как помогает распознать близкие по смыслу или происхождению слова. Так как задача предполагает поиск похожих по смыслу предложений, нормализация - один из основных методов. 
3) Эмбеддинги, обученные на корпусе похожих вопросов. Применены описанные выше методы предобработки текста, которые повысили качество модели. Подбирались гиперпараметры модели.
4) Качество решения зависит от предобработки данных, подхода к формированию эмбеддингов, метода решения задачи. Необходима дальнейшая предобработка данных, учёт частоты встречаемости слов, распределение слов по важности в контексте (например, слова javascript, canvas важнее в контексте вопроса, чем how, I). Также качество зависит от реализации функции rank_candidates, улучшить которую в рамках работы не удалось. Также возможно применение более продвинутого метода ранжирования. 
5) Возможно использование предобученной модели. 